# Look at previous code for GCTA, run with our data, become familiar

In [1]:
library(data.table)

In [2]:
outdir <- "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/gcta_output"
if(!dir.exists(outdir)) dir.create(outdir)

In [3]:
chunk1 <- 1

In [4]:
chunk2 <- 10

## Check out these VMRs. Is this something we need to use, or is the same info in our newer file formats?

In [5]:
# library(data.table)  # For efficient data handling

# args <- commandArgs(trailingOnly=TRUE)

# indir <- "/dcs04/lieber/statsgen/shizhong/AANRI/VMR2/99/caud/aa/"

# setwd(outdir)

# # Initialize lists to store sizes, dimensions, and loading times
# data_sizes <- list()
# data_dims <- list()
# loading_times <- list()

# # Load and analyze VMR files for each chromosome
# for (i in 1:22) {
#     cat("Processing chromosome", i, "...\n")
    
#     # Measure loading time
#     start_time <- Sys.time()
#     load(paste0(indir, "out/chr", i, "_vmr.rda"))
#     end_time <- Sys.time()
#     loading_time <- as.numeric(difftime(end_time, start_time, units = "secs"))
    
#     # Store loading time
#     loading_times[[i]] <- loading_time
    
#     # Print the 5x5 slice of the methylation data and head of VMR data
#     cat("Slice [1:5, 1:5] of meth for chromosome", i, ":\n")
#     print(meth[1:5, 1:5])
#     cat("Head of vmrs for chromosome", i, ":\n")
#     print(head(vmrs))
    
#     # Store the size and dimensions of the data
#     data_sizes[[i]] <- list(
#         meth_size = object.size(meth),
#         vmrs_size = object.size(vmrs)
#     )
#     data_dims[[i]] <- list(
#         meth_dim = dim(meth),
#         vmrs_dim = dim(vmrs)
#     )
    
#     # Combine data across chromosomes
#     if (i == 1) {
#         meth2 <- meth
#         vmrs2 <- vmrs
#     } else {
#         meth2 <- rbind(meth2, meth)
#         vmrs2 <- rbind(vmrs2, vmrs)
#     }
# }

# # Print summary information
# cat("\nSummary of loading times (seconds):\n")
# for (i in 1:22) {
#     cat("Chromosome", i, ":", loading_times[[i]], "seconds\n")
# }

# cat("\nSummary of data sizes (bytes):\n")
# for (i in 1:22) {
#     cat("Chromosome", i, ":\n")
#     cat("  meth size:", data_sizes[[i]]$meth_size, "bytes\n")
#     cat("  vmrs size:", data_sizes[[i]]$vmrs_size, "bytes\n")
# }

# cat("\nSummary of data dimensions:\n")
# for (i in 1:22) {
#     cat("Chromosome", i, ":\n")
#     cat("  meth dimensions:", data_dims[[i]]$meth_dim, "\n")
#     cat("  vmrs dimensions:", data_dims[[i]]$vmrs_dim, "\n")
# }


Interesting how the allele data and methylationd ata appears to be for teh same limited number of sites. We use more SNPs as explanatory variables to compute heritability, right?

## Try with new data

In [6]:
library(data.table)

In [7]:
df <- fread("/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/09.5-OUT_matched_SNP_meth_cov_chunked_JHPCE.csv")

In [8]:
df_row <- 1

In [9]:
#args = commandArgs(trailingOnly=TRUE)

#outdir <- args[1]
#chunk1 <- as.numeric(args[2])
#chunk2 <- as.numeric(args[3])

wind <- c(10000, 100000, 1000000)

#wind <- as.numeric(c("1000","2000","5000","10000","20000","50000","100000","200000","500000"))

# This directory contains vmr files, which are...
#indir <- "/dcs04/lieber/statsgen/shizhong/AANRI/VMR2/99/caud/aa/"

# path to dir containing pgen, psam files for a given population/subpopulation
gwas <- paste0(dirname(df$SNP_data[df_row]), "/")
gcta <- "/dcs04/lieber/statsgen/shizhong/software/gcta/gcta-1.94.1-linux-kernel-3-x86_64/gcta-1.94.1"

outdir <- "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/gcta_output"
if(!dir.exists(outdir)) dir.create(outdir)

setwd(outdir)

In [10]:
gwas

[1] "/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas/"

In [11]:
library(bsseq)

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:data.table’:

    first, second


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname

In [12]:
load(df$methylation_data[df_row])
chunk1 <- 1
chunk2 <- 50

### Get our methylation data in the same format as the earlier `p` matrix (row per sample, column per site)

In [13]:
p <- t(as.matrix(getMeth(BSobj2, type = "smooth", what = "perBase")))

ind <- BSobj2@colData$brnum <- gsub("Br0", "Br", BSobj2@colData$brnum)

ind <- gsub("Br0", "Br", BSobj2@colData$brnum)
id <- ind

rownames(p) <- ind

covs <- fread(df$cov_file[df_row])
covs$Sex[covs$Sex=="M"] <- 0
covs$Sex[covs$Sex=="F"] <- 1

covs$Dx[covs$Dx=="Control"] <- 0
covs$Dx[covs$Dx=="SCZ"] <- 1

missing_ids <- ind[!ind %in% covs$ID]
if (length(missing_ids) > 0) {
  cat("Missing covariates for IDs:", paste(missing_ids, collapse = ", "), "\n")
  ind <- ind[ind %in% covs$ID]
}
covs <- covs[match(ind, covs$ID), ]
p <- p[match(ind, rownames(p)), ]

id <- colData(BSobj2)$ID[which(colData(BSobj2)$brnum %in% ind)]

covs <- cbind(0, covs)
colnames(covs)[1] <- "intercept"

write.table(ind,"id",col.names=F,row.names=F,quote=F,sep="\t")

write.table(covs,"covs",col.names=F,row.names=F,quote=F,sep="\t")

chr <- unfactor(unique(seqnames(rowRanges(BSobj2))))
if(length(chr) > 1) stop ("Should be just one chromosome per BSobj")
chr <- gsub("chr","",chr)

CpG_positions <- start(ranges(granges(BSobj2)))

#### With multithreading

In [14]:
# Sys.time()

In [15]:
# # loop over vmr between two chunks
# res <- c()
# for(i in chunk1:chunk2){
# 	cat(i,"\n")
# 	chr <- gsub("chr","",chr)
# 	# loop over each window size
# 	for(w in 1:length(wind)){
#         # plink subset
#         p1 <- ifelse(CpG_positions[i] - wind[w] > 0, CpG_positions[i] - wind[w],0)
#         p2 <- CpG_positions[i] + wind[w]
#         gwas_prefix <- paste0(gwas,"libd_chr",chr)
#         command <- paste("/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/plink2 --pfile ", gwas_prefix, "--silent --keep id",
#                          "--chr ",chr,
#                          "--from-bp",p1,"--to-bp",p2,
#                          "--snps-only 'just-acgt' --make-bed --out temp",
#                          sep=" ")
#         system(command)	
#         if(!file.exists("temp.bim")){
#             next;
#         }
#         # phenotype file 
#         #pheno <- cbind(0,id,p[,i])
#         pheno <- cbind(0,ind,p[,i])
#         write.table(pheno,"pheno",col.names=F,row.names=F,quote=F,sep="\t")
#         # grm
#         command <- paste(gcta, "--bfile temp --make-grm-bin --out temp", sep=" ")
#         system(command)
#         # h2 estimation
#         command <- paste(gcta, "--reml --grm-bin temp --pheno pheno --mpheno 1 --qcovar covs --out temp", sep=" ")
#         system(command)
#         # collect results
#         if(!file.exists("temp.hsq")){
#             next;
#         }
#         temp <- read.table("temp.hsq",header=T, fill=TRUE)
#         temp$site <- paste0("chr",chr,"_",CpG_positions[i])
#         temp$wind <- wind[w]
#         res <- rbind(res,temp)
#         # remove temp files
#         system("rm temp*")
#         }
	
# }
# write.table(res,"res.txt",col.names=T,row.names=F,quote=F,sep="\t") 

In [16]:
# Sys.time()

#### Single-threaded

In [17]:
Sys.time() # 2024-07-09 10:01:27 EDT

[1] "2024-08-28 12:48:08 EDT"

In [18]:
getwd()

[1] "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/gcta_output"

In [19]:
w

ERROR: Error in eval(expr, envir, enclos): object 'w' not found


In [ ]:
?system()

In [ ]:
intern <- TRUE

In [ ]:
library(data.table)

res <- NULL

benchmark <- TRUE
if (benchmark) {
    time_measurements <- list()
}

for (i in chunk1:chunk2) {
    chr <- gsub("chr", "", chr)
    print(i)

    for (w in 1:length(wind)) {
        print(wind[w])
        if (benchmark) {
            start_time <- Sys.time()
        }

        # PLINK subset
        p1 <- ifelse(CpG_positions[i] - wind[w] > 0, CpG_positions[i] - wind[w], 0)
        p2 <- CpG_positions[i] + wind[w]
        gwas_prefix <- paste0(gwas, "libd_chr", chr)
        command <- paste("/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/plink2 --pfile ", gwas_prefix, "--silent --keep id",
                         "--chr", chr,
                         "--from-bp", p1, "--to-bp", p2,
                         "--snps-only 'just-acgt' --make-bed --threads 1 --out temp",
                         sep = " ")
        system(command)
        
        if (benchmark) {
            plink_time <- Sys.time() - start_time
            start_time <- Sys.time()
        }
        
        if (!file.exists("temp.bim")) {
            print("skip this window for this CpG")
            next
        }

        # Phenotype file
        pheno <- cbind(0, ind, p[, i])
        write.table(pheno, "pheno", col.names = FALSE, row.names = FALSE, quote = FALSE, sep = "\t")

        # GRM
        command <- paste(gcta, "--bfile temp --make-grm-bin --thread-num 1 --out temp", sep = " ")
        c1_out <- system(command, intern = intern)
        if(intern) print(c1_out)

        if (benchmark) {
            grm_time <- Sys.time() - start_time
            start_time <- Sys.time()
        }

        # Heritability estimation
        command <- paste(gcta, "--reml --grm-bin temp --pheno pheno --mpheno 1 --qcovar covs --thread-num 1 --out temp", sep = " ")
        c2_out <- system(command, intern = intern)
        if(intern) print(c2_out)

        if (benchmark) {
            reml_time <- Sys.time() - start_time
        }

        if (!file.exists("temp.hsq")) {
            stop("WTF")
            next
        }

        # Read and reshape temp
        # Assuming temp_before_transpose is already created as shown
        temp_before_transpose <- fread("temp.hsq", header = TRUE, fill = TRUE)

        # Create a new data frame `temp2` by manually extracting the values using column numbers
        temp2 <- list(
          V_G = temp_before_transpose[[1, 2]],
          SE_V_G = temp_before_transpose[[1, 3]],
          V_e = temp_before_transpose[[2, 2]],
          SE_V_e = temp_before_transpose[[2, 3]],
          Vp = temp_before_transpose[[3, 2]],
          SE_Vp = temp_before_transpose[[3, 3]],
          V_G_Vp = temp_before_transpose[[4, 2]],
          SE_V_G_Vp = temp_before_transpose[[4, 3]],
          logL = temp_before_transpose[[5, 2]],
          logL0 = temp_before_transpose[[6, 2]],
          LRT = temp_before_transpose[[7, 2]],
          df = temp_before_transpose[[8, 2]],
          Pval = temp_before_transpose[[9, 2]],
          n = temp_before_transpose[[10, 2]]
        )
        
        
        
        temp2$site <- paste0("chr", chr, "_", CpG_positions[i])
        temp2$wind <- wind[w]
        print(as.data.frame(temp2))
        
        #if(temp2$n <= 1) stop("whatttt")
        #if(temp2$V_G == 1) stop("what???")
        #if(nrow(temp2) > 1) stop("huhh")
        # After creating temp2
        if (is.null(temp2) || temp2$n <= 1) {
            # Log this issue for further investigation if necessary
            cat("Skipping invalid row with n =", temp2$n, "or incorrect row count\n")
            next  # Skip this iteration
        }

        
        if (is.null(res)) {
            res <- temp2
        } else {
            res <- rbind(res, temp2)
        }

        # Remove temp files
        system("rm temp*")
        
        # Collect timing data if benchmarking
        if (benchmark) {
            time_measurements[[paste0("chr", chr, "_pos", CpG_positions[i], "_wind", wind[w])]] <- list(plink_time, grm_time, reml_time)
        }
    }
    cat("\n")
}

# Convert timing list to a data.table and write to file if benchmarking
if (benchmark) {
    timing_dt <- rbindlist(lapply(names(time_measurements), function(x) {
        list(ID = x, plink_time = time_measurements[[x]][[1]], grm_time = time_measurements[[x]][[2]], reml_time = time_measurements[[x]][[3]])
    }), use.names = TRUE, fill = TRUE, idcol = "ID")
    fwrite(timing_dt, "timing_measurements.txt", col.names = TRUE, sep = "\t")
}

In [ ]:
temp2 # after reorganizing gcta output, ready to append to results

In [ ]:
temp_before_transpose # output direct from gcta

In [ ]:
temp2

In [ ]:
# # Convert timing list to a data.table and write to file if benchmarking
# if (benchmark) {
#     timing_dt <- rbindlist(lapply(names(time_measurements), function(x) {
#         list(ID = x, plink_time = time_measurements[[x]][[1]], grm_time = time_measurements[[x]][[2]], reml_time = time_measurements[[x]][[3]])
#     }), use.names = TRUE, fill = TRUE, idcol = "ID")
#     fwrite(timing_dt, "timing_measurements.txt", col.names = TRUE, sep = "\t")
# }

In [ ]:
Sys.time() # 2024-07-09 10:22:36 EDT

In [20]:
getwd()

[1] "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/gcta_output"

In [30]:
levels(factor(res$wind))

character(0)

In [29]:
table(res$wind)

< table of extent 0 >

In [36]:
res

V_G      SE_V_G   V_e      SE_V_e   Vp       SE_Vp    V_G_Vp   SE_V_G_Vp
res      5e-06  1.2e-05 0.000265  3.2e-05 0.000269  3.2e-05 0.017715 0.044797 
temp2    4e-06  1.2e-05 0.000272  3.3e-05 0.000277  3.2e-05 0.016255 0.043039 
temp2    5e-06  1.3e-05 0.000278  3.4e-05 0.000284  3.3e-05  0.01904  0.04437 
temp2    6e-06  1.3e-05 0.000284  3.5e-05  0.00029  3.4e-05 0.020028  0.04478 
temp2    8e-06  1.6e-05 0.000316  3.9e-05 0.000324  3.8e-05 0.025539  0.04751 
temp2  1.5e-05  2.3e-05 0.000422  5.2e-05 0.000437  5.1e-05 0.035222 0.051507 
temp2  1.9e-05  2.7e-05 0.000476  5.8e-05 0.000495  5.8e-05 0.039206 0.053202 
temp2  2.9e-05  3.5e-05 0.000572    7e-05 0.000601  7.1e-05 0.047921 0.057098 
temp2  4.5e-05  4.8e-05 0.000695  8.6e-05  0.00074  8.8e-05 0.060861 0.062803 
temp2  5.1e-05  5.2e-05 0.000734  9.1e-05 0.000785  9.4e-05 0.064429 0.064348 
temp2 0.000127 0.000111 0.001253 0.000157  0.00138 0.000169 0.092272 0.076182 
temp2  0.00016 0.000136  0.00147 0.000185  0.00163 0.000201 0.098201 0.078634 
temp2 0.000169 0.000142 0.001528 0.000192 0.001696 0.000209 0.099451  0.07914 
temp2 0.000174 0.000146 0.001562 0.000197 0.001736 0.000214 0.100142 0.079416 
temp2 0.000182 0.000153  0.00162 0.000204 0.001803 0.000223 0.101209 0.079836 
temp2 0.000185 0.000155 0.001638 0.000207 0.001823 0.000225 0.101513 0.079953 
temp2 0.000223 0.000185 0.001982 0.000249 0.002204 0.000272  0.10097 0.078935 
temp2 0.000216 0.000182 0.002005 0.000252 0.002221 0.000273 0.097451 0.077405 
temp2 0.000215 0.000181  0.00201 0.000252 0.002224 0.000273 0.096493 0.077003 
temp2 0.000214 0.000181 0.002011 0.000253 0.002225 0.000273 0.096214 0.076886 
temp2 0.000208 0.000178 0.002026 0.000254 0.002233 0.000273 0.092934 0.075534 
temp2 0.000183 0.000167 0.002067 0.000258  0.00225 0.000273 0.081194 0.070846 
temp2 0.000181 0.000166 0.002071 0.000258 0.002252 0.000273 0.080231 0.070461 
temp2 0.000177 0.000164 0.002078 0.000259 0.002255 0.000273 0.078612 0.069814 
temp2 0.000169 0.000161 0.002097 0.000261 0.002266 0.000273 0.074532 0.068177 
temp2 0.000146 0.000151 0.002162 0.000268 0.002308 0.000276 0.063044 0.063528 
temp2  0.00014 0.000149 0.002181  0.00027 0.002321 0.000277  0.06029 0.062403 
temp2 0.000139 0.000149 0.002184  0.00027 0.002323 0.000277 0.059888 0.062239 
temp2 0.000135 0.000147   0.0022 0.000272 0.002335 0.000278 0.057614 0.061307 
temp2 0.000132 0.000146 0.002211 0.000273 0.002343 0.000279 0.056151 0.060707 
temp2 0.000127 0.000144  0.00223 0.000276 0.002357  0.00028 0.053771 0.059727 
temp2 0.000118 0.000141 0.002266  0.00028 0.002384 0.000282 0.049387 0.057913 
temp2 0.000112 0.000139 0.002291 0.000282 0.002403 0.000284 0.046611 0.056737 
temp2 0.000108 0.000138 0.002307 0.000284 0.002415 0.000285 0.044818 0.055984 
temp2  7.4e-05 0.000126 0.002465 0.000302 0.002539 0.000298 0.029137 0.049115 
temp2  7.1e-05 0.000125 0.002479 0.000304  0.00255 0.000299 0.027782 0.048487 
temp2    6e-05  0.00012  0.00253  0.00031 0.002589 0.000303 0.022982 0.046195 
temp2    5e-05 0.000116  0.00257 0.000314  0.00262 0.000306 0.019042 0.044223 
temp2  4.5e-05 0.000114 0.002591 0.000317 0.002636 0.000308 0.017007 0.043168 
temp2  3.1e-05 0.000108 0.002644 0.000322 0.002675 0.000312 0.011626 0.040231 
temp2  1.4e-05  9.9e-05 0.002701 0.000329 0.002715 0.000317 0.005186 0.036444 
temp2        0  9.7e-05 0.002768 0.000336 0.002768 0.000323    1e-06 0.035048 
temp2        0  9.9e-05 0.002774 0.000337 0.002774 0.000324    1e-06 0.035808 
temp2        0  9.9e-05 0.002774 0.000337 0.002774 0.000324    1e-06 0.035841 
temp2        0    1e-04 0.002774 0.000337 0.002774 0.000324    1e-06 0.035907 
temp2        0 0.000101 0.002775 0.000337 0.002775 0.000324    1e-06 0.036222 
temp2        0 0.000111 0.002748 0.000334 0.002748 0.000321    1e-06 0.040518 
temp2        0 0.000115 0.002743 0.000333 0.002743  0.00032    1e-06 0.041918 
temp2        0 0.000115 0.002864 0.000348 0.002864 0.000334    1e-06 0.040266 
temp2        0 0.000114 0.003017 0.000366 0.003017 0.000

In [33]:
tail(res)

V_G SE_V_G   V_e      SE_V_e   Vp       SE_Vp    V_G_Vp SE_V_G_Vp logL   
temp2 0      1e-04 0.002774 0.000337 0.002774 0.000324 1e-06  0.035907  278.238
temp2 0   0.000101 0.002775 0.000337 0.002775 0.000324 1e-06  0.036222  278.204
temp2 0   0.000111 0.002748 0.000334 0.002748 0.000321 1e-06  0.040518  278.739
temp2 0   0.000115 0.002743 0.000333 0.002743  0.00032 1e-06  0.041918  278.798
temp2 0   0.000115 0.002864 0.000348 0.002864 0.000334 1e-06  0.040266  275.721
temp2 0   0.000114 0.003017 0.000366 0.003017 0.000352 1e-06  0.037944  271.993
      logL0   LRT df Pval n   site        wind 
temp2 278.238 0   1  0.5  164 chr1_796799 1e+06
temp2 278.204 0   1  0.5  164 chr1_796818 1e+06
temp2 278.739 0   1  0.5  164 chr1_797103 1e+06
temp2 278.799 0   1  0.5  164 chr1_797308 1e+06
temp2 275.721 0   1  0.5  164 chr1_797590 1e+06
temp2 271.993 0   1  0.5  164 chr1_797744 1e+06

In [34]:
temp2

$V_G
[1] 0

$SE_V_G
[1] 0.000114

$V_e
[1] 0.003017

$SE_V_e
[1] 0.000366

$Vp
[1] 0.003017

$SE_Vp
[1] 0.000352

$V_G_Vp
[1] 1e-06

$SE_V_G_Vp
[1] 0.037944

$logL
[1] 271.993

$logL0
[1] 271.993

$LRT
[1] 0

$df
[1] 1

$Pval
[1] 0.5

$n
[1] 164

$site
[1] "chr1_797744"

$wind
[1] 1e+06

In [35]:
temp_before_transpose

Source,Variance,SE
<chr>,<dbl>,<dbl>
V(G),0.000000,0.000114
V(e),0.003017,0.000366
Vp,0.003017,0.000352
V(G)/Vp,0.000001,0.037944
logL,271.993000,NA
logL0,271.993000,NA
LRT,0.000000,NA
df,1.000000,NA
Pval,0.500000,NA


In [32]:
res[30:40, ]

V_G      SE_V_G   V_e      SE_V_e   Vp       SE_Vp    V_G_Vp   SE_V_G_Vp
temp2 0.000132 0.000146 0.002211 0.000273 0.002343 0.000279 0.056151 0.060707 
temp2 0.000127 0.000144  0.00223 0.000276 0.002357  0.00028 0.053771 0.059727 
temp2 0.000118 0.000141 0.002266  0.00028 0.002384 0.000282 0.049387 0.057913 
temp2 0.000112 0.000139 0.002291 0.000282 0.002403 0.000284 0.046611 0.056737 
temp2 0.000108 0.000138 0.002307 0.000284 0.002415 0.000285 0.044818 0.055984 
temp2  7.4e-05 0.000126 0.002465 0.000302 0.002539 0.000298 0.029137 0.049115 
temp2  7.1e-05 0.000125 0.002479 0.000304  0.00255 0.000299 0.027782 0.048487 
temp2    6e-05  0.00012  0.00253  0.00031 0.002589 0.000303 0.022982 0.046195 
temp2    5e-05 0.000116  0.00257 0.000314  0.00262 0.000306 0.019042 0.044223 
temp2  4.5e-05 0.000114 0.002591 0.000317 0.002636 0.000308 0.017007 0.043168 
temp2  3.1e-05 0.000108 0.002644 0.000322 0.002675 0.000312 0.011626 0.040231 
      logL    logL0   LRT   df Pval     n   site        wind 
temp2 291.646  290.59 2.112 1  0.073062 164 chr1_795738 1e+06
temp2  291.15 290.179 1.943 1  0.081678 164 chr1_795774 1e+06
temp2 290.181 289.358 1.645 1   0.09979 164 chr1_795841 1e+06
temp2 289.534   288.8 1.468 1   0.11286 164 chr1_795884 1e+06
temp2 289.104 288.425 1.358 1   0.12198 164 chr1_795912 1e+06
temp2 285.101 284.817 0.568 1   0.22547 164 chr1_796163 1e+06
temp2 284.753 284.495 0.516 1   0.23636 164 chr1_796185 1e+06
temp2 283.545  283.37  0.35 1    0.2771 164 chr1_796263 1e+06
temp2 282.605 282.486 0.238 1   0.31281 164 chr1_796327 1e+06
temp2 282.142 282.048 0.189 1   0.33193 164 chr1_796360 1e+06
temp2 281.009 280.966 0.087 1    0.3842 164 chr1_796447 1e+06

In [ ]:
# number seconds to run

In [ ]:
n_sec <- (21*60)+9
n_min <- n_sec/60
n_hr <- n_min/60
print(n_hr)

In [ ]:
core_hr_per_cpg <- n_hr/1000

In [ ]:
core_hr_per_cpg*26000000

In [43]:
chunk2 <- 5000

In [44]:
Sys.time()

[1] "2024-08-22 12:07:23 EDT"

In [ ]:
# loop over vmr between two chunks
res <- c()
for(i in chunk1:chunk2){
	cat(i,"\n")
	chr <- gsub("chr","",chr)
	# loop over each window size
	for(w in 1:length(wind)){
        # plink subset
        p1 <- ifelse(CpG_positions[i] - wind[w] > 0, CpG_positions[i] - wind[w],0)
        p2 <- CpG_positions[i] + wind[w]
        gwas_prefix <- paste0(gwas,"libd_chr",chr)
        command <- paste("/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts/plink2 --pfile ", gwas_prefix, "--silent --keep id",
                         "--chr ",chr,
                         "--from-bp",p1,"--to-bp",p2,
                         "--snps-only 'just-acgt' --make-bed --threads 1 --out temp",
                         sep=" ")
        system(command)	
        if(!file.exists("temp.bim")){
            next;
        }
        # phenotype file 
        #pheno <- cbind(0,id,p[,i])
        pheno <- cbind(0,ind,p[,i])
        write.table(pheno,"pheno",col.names=F,row.names=F,quote=F,sep="\t")
        # grm
        command <- paste(gcta, "--bfile temp --make-grm-bin --thread-num 1 --out temp", sep=" ")
        system(command)
        # h2 estimation
        command <- paste(gcta, "--reml --grm-bin temp --pheno pheno --mpheno 1 --qcovar covs --thread-num 1 --out temp", sep=" ")
        system(command)
        # collect results
        if(!file.exists("temp.hsq")){
            next;
        }
        temp <- read.table("temp.hsq",header=T, fill=TRUE)
        temp$site <- paste0("chr",chr,"_",CpG_positions[i])
        temp$wind <- wind[w]
        res <- rbind(res,temp)
        # remove temp files
        system("rm temp*")
        }
	
}
write.table(res,"res-singlethread.txt",col.names=T,row.names=F,quote=F,sep="\t") 

In [ ]:
Sys.time()

5000 tests in 1hr54min47sec. 1.9136hr

In [ ]:
(1.913/5000)*2.6*10^6

In [ ]:
(1.913/5000)*26*10^6 # Estimated runtime for all

In [ ]:
|